<!--
SPDX-FileCopyrightText: 2025 Silva F. F. <fernandoferreira.silva42@usp.br>
SPDX-FileCopyrightText: 2025 Monaco F. J. <monaco@usp.br>

SPDX-License-Identifier: GPL-3.0-or-later
-->

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opensciware/moeabench/blob/main/examples/example-05.ipynb)

# MoeaBench Example 05: Custom MOP

How to define your own optimization problem by subclassing `BaseMOP`.

In [ ]:
# Install MoeaBench
!pip install --quiet git+https://github.com/opensciware/moeabench.git

In [ ]:
import MoeaBench as mb
import numpy as np

# 1. Define Custom MOP Class
class MyProblem(mb.mops.BaseMOP):
    """
    A simple custom bi-objective problem (ZDT1-like).
    """
    def __init__(self):
        # Initialize: 2 Objectives (M), 30 Variables (N)
        # Bounds: 0.0 to 1.0
        super().__init__(M=2, N=30, xl=0.0, xu=1.0)

    def evaluation(self, X, n_ieq_constr=0):
        # Vectorized evaluation logic
        # X shape: (population_size, N_vars)
        
        # Objective 1: Just the first variable
        f1 = X[:, 0]
        
        # Objective 2: Transformation of the others
        g = 1 + 9 * np.sum(X[:, 1:], axis=1) / (self.N - 1)
        f2 = g * (1 - np.sqrt(f1 / g))
        
        # Return dictionary with 'F' (Objectives)
        return {'F': np.column_stack([f1, f2])}

In [ ]:
# 2. Run Experiment
my_problem = MyProblem()

exp = mb.experiment()
exp.mop = my_problem
exp.moea = mb.moeas.NSGA3(population=100, generations=50)
exp.name = "MyCustomProblem"

exp.run()

# 3. Visualize (Auto-detects 2D problem and plots appropriately)
mb.spaceplot(exp, title="Pareto Front of MyCustomProblem")